# Run Moatless Tools

First, index the codebase in a vector store.

Set `repo_dir` to the path of the repository you want to index.

In [4]:
from moatless.index import CodeIndex, IndexSettings
from moatless import FileRepository, Workspace

# An OPENAI_API_KEY is required to use the OpenAI Models
model = "gpt-4o-2024-05-13"
index_settings = IndexSettings(
    embed_model="text-embedding-3-small"
)

repo_dir = "/tmp/moatless-tools"
file_repo = FileRepository(repo_path=repo_dir)

code_index = CodeIndex(file_repo=file_repo, settings=index_settings)
nodes, tokens = code_index.run_ingestion()

print(f"Indexed {nodes} nodes and {tokens} tokens")

workspace = Workspace(file_repo=file_repo, code_index=code_index)

Parsing nodes:   0%|          | 0/48 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/198 [00:00<?, ?it/s]

Indexed 198 nodes and 65001 tokens


Then use the `SearchLoop` to find the relevant code.

In [5]:
from moatless.transitions import search_transitions
from moatless import AgenticLoop

instructions = "Remove the token limit check from the completion function"

transitions = search_transitions()
search_loop = AgenticLoop(transitions, workspace=workspace)

search_response = search_loop.run(instructions)
print(search_response.message)

print(workspace.file_context.create_prompt())

The relevant code span for the completion function is found in moatless/llm/completion.py. The token limit check is present in the line `if tokens > Settings.max_message_tokens:`. This is the code that needs to be modified to remove the token limit check.


moatless/llm/completion.py
```



def completion(
    model: str,
    messages: List,
    max_tokens: int = 1000,
    temperature: float = 0.0,
    trace_name: str = "moatless-agent",
    stop: Optional[List[str]] = None,
    generation_name: Optional[str] = None,
    tools: Optional[List[Dict[str, Any]]] = None,
) -> litellm.ModelResponse:
    if len(messages) == 0:
        raise ValueError("At least one message is required.")

    global _trace_metadata, _mock_response
    metadata = {}
    metadata.update(_trace_metadata)

    if generation_name:
        metadata["generation_name"] = generation_name

    metadata["trace_name"] = trace_name

    tokens = token_counter(messages=messages[-1:])
    if tokens > Settings.max_message_to

Execute the `CodeLoop` to apply the changes.

In [6]:
from moatless.transitions import code_transitions

code_loop = AgenticLoop(transitions=code_transitions(), workspace=workspace)
code_response = code_loop.run(instructions)

print(f"Response: {code_response.message}")

Response: The token limit check has been successfully removed from the `completion` function in moatless/llm/completion.py.


Run a `$ git diff` to see the changes.

In [7]:
import subprocess

output = subprocess.run(
      ["git", "diff"],
      capture_output=True,
      text=True,
      cwd=repo_dir,
)

print(output.stdout)

diff --git a/moatless/llm/completion.py b/moatless/llm/completion.py
index 2c95e47..a926948 100644
--- a/moatless/llm/completion.py
+++ b/moatless/llm/completion.py
@@ -48,10 +48,6 @@ def completion(
 
     metadata["trace_name"] = trace_name
 
-    tokens = token_counter(messages=messages[-1:])
-    if tokens > Settings.max_message_tokens:
-        raise ValueError(f"Too many tokens in the new message: {tokens}")
-
     response = litellm.completion(
         model=model,
         max_tokens=max_tokens,

